# Code lấy dữ liệu từ BaseVN

## 1. Kết nối notebook với Google Drive
Bước này có thể bỏ qua nếu không muốn lưu vào Drive. Tuy nhiên các output files sẽ mất sau khi restart session

In [ ]:
"""
Chạy đoạn code dưới đây để kết nối notebook với Google Drive
"""
from google.colab import drive
drive.mount('/content/drive')

## 2. Gửi request và lấy dữ liệu form

In [ ]:
import requests
import pandas as pd
import os
import json
from tqdm import tqdm
import numpy as np

def get_data(url: str = "https://workflow.base.vn/extapi/v1/workflow/jobs",
             payload: dict = {}):

    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    response = requests.post(url, data=payload, headers=headers)
    print("Status code:", response.status_code)
    try:
        return response.json()
    except ValueError:
        return {}


if __name__ == "__main__":
    access_token = "ADD YOUR TOKEN HERE"
    output_folder_path = "/content/drive/MyDrive/outputs" # Thay output path bằng folder chứa file kết quả

    # Sửa đổi payload các trường cần lấy data theo format {"key1": "value1", "key2": "value2", ...}
    payload = {
        "access_token": f"{access_token}",
        "id": "13080",
        "created_from": "8/3/2025",
        "created_to": "8/4/2025"
    }
    # --------------------------------------------------------- #
    data = get_data(payload=payload)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    full_values = []
    max_length_headers = []
    full_data = []
    for job in tqdm(data["jobs"]):
        form = job["form_origin"]
        headers, values = [], []
        form_dict = {}
        for item in form:
            # print(item)
            name = item['name']
            value = item['value']
            form_dict[name] = value
            headers.append(name)
            # values.append(value)
        if len(headers) > len(max_length_headers):
            max_length_headers = headers.copy()

        full_data.append(form_dict)
        full_values.append(values)
    

    all_keys = set()
    for sample in full_data:
        all_keys.update(sample.keys())

    df = pd.DataFrame(full_data, columns=max_length_headers)
    df.fillna('', inplace=True)
    # --------------------------------------------------------- #

    # full_output.xlsx là tên file cần lưu
    output_file_name = input("Output filename: ") or "full_output.xlsx"
    if not ".xlsx" in output_file_name:
        output_file_name += ".xlsx"
    df.to_excel(f'{output_folder_path}/{output_file_name}', index=False)

    print(f"\nOutput file is saved at {output_folder_path}/{output_file_name}")